# 🤖 AI-Powered Customer Support System

This project demonstrates how to build an **AI-powered customer support workflow**  
using **LangGraph**, **LangChain**, and **Google Gemini (via `langchain-google-genai`)**.

---

### 🔧 What This Notebook Covers
1. **Environment Setup** – Loading API keys securely using `.env`
2. **LLM Configuration** – Setting up Google Gemini
3. **State Definition** – Using `TypedDict` for structured state management
4. **Node Functions** – Categorization, Sentiment Analysis, and Response Generation
5. **Workflow Orchestration** – Using `StateGraph` to route queries
6. **Testing & Debugging** – Running sample queries and displaying results

---

### 🎯 Goal
Automatically:
- Categorize customer queries (`Technical`, `Billing`, `General`)
- Detect sentiment (`Positive`, `Negative`, `Neutral`)
- Generate AI-based responses
- Escalate negative queries to human agents when needed

This is a **production-ready notebook** with modular code and debug-friendly logging.


In [21]:
from typing import TypedDict, Dict
from langgraph.graph import StateGraph,END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
import os
from IPython.display import display,Image

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load Environment Variables

In [22]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found. Please add it to your .env file.")
print("✅ Environment variables loaded successfully")

✅ Environment variables loaded successfully


# Configure LLM

In [23]:
# Configure Gemini model for response generation
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=google_api_key,
    temperature=0.7,
)

print("✅ LLM configured successfully")

✅ LLM configured successfully


# Define state structure

In [24]:
# a dictionary with specific key types and value types for type checking 
class State(TypedDict):
  query: str
  category: str
  sentiment: str
  response: str

# Define Nodes

In [25]:
def categorize(state: State) -> State:
    """Technical, Billing, General"""
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following customer query into one of these categories: Technical, Billing, General.\n\nquery:{query}\n\n"

    )

    chain = prompt | llm

    category = chain.invoke({"query":state['query']}).content.strip()
    return {**state, "category":category}



In [26]:
def analyze_sentiment(state:State)-> State:
    prompt = ChatPromptTemplate.from_template(
        "Analyze the sentiment of the following customer query"
        "response with 'Positive', 'Negative', 'Neutral'.\n\n"
        "query:{query}\n\n")
    chain = prompt | llm
    sentiment = chain.invoke({"query":state['query']}).content.strip()
    print(sentiment)
    return {**state,"sentiment":sentiment}

In [27]:

def handle_technical(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a detailed but friendly technical support response to the following query: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content.strip()
    print(f"🛠 Response: {response}")
    return {**state, "response": response}

In [28]:

def handle_billing(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a billing support response to the following query: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content.strip()
    return {**state, "response": response}

In [29]:
def handle_general(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "Provide a general support response to the following query: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content.strip()
    return {**state, "response": response}

In [30]:
def escalate_to_human(state: State) -> State:
    return {**state, "response": "Your query has been escalated to a human agent."}


# Define Routing & Workflow

In [31]:
def route_query(state: State) -> str:
    if state["sentiment"].lower() == "negative":
        return "escalate_to_human"
    elif "Technical" in state["category"]:
        return "handle_technical"
    elif "Billing" in state["category"]:
        return "handle_billing"
    else:
        return "handle_general"

In [32]:
workflow = StateGraph(State)
workflow.add_node("categorize",categorize)
workflow.add_node("analyze_sentiment",analyze_sentiment)
workflow.add_node("handle_technical",handle_technical)
workflow.add_node("handle_billing",handle_billing)
workflow.add_node("handle_general",handle_general)
workflow.add_node("escalate_to_human",escalate_to_human)



In [33]:
workflow.add_edge("categorize","analyze_sentiment")
workflow.add_conditional_edges("analyze_sentiment",
                               route_query,{
                                   "handle_technical":"handle_technical",
                                   "handle_billing":"handle_billing",
                                   "handle_general":"handle_general",
                                   "escalate_to_human":"escalate_to_human"
                               })
workflow.add_edge("handle_technical",END)
workflow.add_edge("handle_billing",END)
workflow.add_edge("handle_general",END)
workflow.add_edge("escalate_to_human",END)

workflow.set_entry_point("categorize")

app = workflow.compile()


# Helper Function & Test

In [36]:
def run_customer_support(query: str) -> Dict[str, str]:
    results = app.invoke({"query": query})
    return {
        "category": results.get("category", ""),
        "sentiment": results.get("sentiment", ""),
        "response": results.get("response", "")
    }

# Test with a sample query
result = run_customer_support("My internet connection is super slow, can you help me?")
result


**Negative**

The customer's query "My internet connection is super slow, can you help me?" expresses a problem and dissatisfaction, indicating a negative sentiment.
🛠 Response: Hi there! Oh no, a slow internet connection is incredibly frustrating, especially when you're trying to get things done or just relax. I'd be happy to help you troubleshoot this and get your speed back up!

There are many reasons why an internet connection might slow down, so let's go through some common steps to pinpoint and fix the issue.

To help us narrow down the cause, could you tell me a little more about what's happening?

*   **Is it slow on ALL your devices (laptop, phone, tablet, smart TV), or just one specific device?**
*   **Is it slow all the time, or only at certain times of the day?**
*   **Is it slow on ALL websites and apps, or just specific ones (e.g., streaming services, online games)?**
*   **Have you noticed any recent changes, like new devices, new software, or new internet service plan?*

{'category': 'This query falls under the **Technical** category.\n\nThe customer is experiencing a performance issue with their internet connection, which requires technical troubleshooting to diagnose and resolve.',
 'sentiment': '**Negative**\n\nThe customer\'s query "My internet connection is super slow, can you help me?" expresses a problem and dissatisfaction, indicating a negative sentiment.',
 'response': 'Hi there! Oh no, a slow internet connection is incredibly frustrating, especially when you\'re trying to get things done or just relax. I\'d be happy to help you troubleshoot this and get your speed back up!\n\nThere are many reasons why an internet connection might slow down, so let\'s go through some common steps to pinpoint and fix the issue.\n\nTo help us narrow down the cause, could you tell me a little more about what\'s happening?\n\n*   **Is it slow on ALL your devices (laptop, phone, tablet, smart TV), or just one specific device?**\n*   **Is it slow all the time, or 

In [37]:
from IPython.display import display, Markdown

def display_result(result: Dict[str, str]):
    md = f"""
**🔖 Category:** {result['category']}
**💡 Sentiment:** {result['sentiment']}

**📝 Response:**  
{result['response']}
"""
    display(Markdown(md))

display_result(result)



**🔖 Category:** This query falls under the **Technical** category.

The customer is experiencing a performance issue with their internet connection, which requires technical troubleshooting to diagnose and resolve.
**💡 Sentiment:** **Negative**

The customer's query "My internet connection is super slow, can you help me?" expresses a problem and dissatisfaction, indicating a negative sentiment.

**📝 Response:**  
Hi there! Oh no, a slow internet connection is incredibly frustrating, especially when you're trying to get things done or just relax. I'd be happy to help you troubleshoot this and get your speed back up!

There are many reasons why an internet connection might slow down, so let's go through some common steps to pinpoint and fix the issue.

To help us narrow down the cause, could you tell me a little more about what's happening?

*   **Is it slow on ALL your devices (laptop, phone, tablet, smart TV), or just one specific device?**
*   **Is it slow all the time, or only at certain times of the day?**
*   **Is it slow on ALL websites and apps, or just specific ones (e.g., streaming services, online games)?**
*   **Have you noticed any recent changes, like new devices, new software, or new internet service plan?**

While you're thinking about that, here are the most common troubleshooting steps you can try right away:

---

### **Step 1: The Classic Reboot (Modem and Router)**

This is the golden rule of tech support for a reason – it often fixes a surprising number of issues by clearing out temporary glitches and refreshing your connection.

1.  **Unplug your modem** (the device that connects to your internet line, often provided by your ISP).
2.  **Unplug your Wi-Fi router** (the device that broadcasts your Wi-Fi signal, sometimes combined with the modem).
3.  **Wait about 30-60 seconds.** This allows the devices to fully power down and clear their memory.
4.  **Plug your modem back in first.** Wait for all its lights to stabilize (this can take a few minutes).
5.  **Once the modem lights are stable, plug your router back in.** Wait for its lights to stabilize as well.
6.  **Test your internet connection** after everything is fully powered up.

---

### **Step 2: Run a Speed Test**

Let's get a measurable idea of what "super slow" means.

1.  Go to a reliable speed test website like **Ookla Speedtest (speedtest.net)** or **Google's Speed Test (just search "speed test" on Google)**.
2.  **Run the test** and make a note of your **Download Speed**, **Upload Speed**, and **Ping** (latency).
3.  **Compare these results to the speed you're paying for** from your Internet Service Provider (ISP). Keep in mind that Wi-Fi speeds are often a bit lower than wired connections, but it gives us a good baseline.

---

### **Step 3: Check Your Wi-Fi Signal & Placement**

Wi-Fi signal strength and interference can significantly impact your speed.

1.  **Move Closer to Your Router:** Try connecting to the internet with your device while sitting very close to your Wi-Fi router. If the speed significantly improves, your issue might be related to Wi-Fi signal strength or interference.
2.  **Router Placement:**
    *   **Central Location:** Your router should ideally be in a central location in your home, not tucked away in a corner or closet.
    *   **Elevated:** Place it on a shelf or table, not on the floor.
    *   **Away from Obstacles:** Keep it away from large metal objects, thick walls, and other electronics (like microwaves, cordless phones, baby monitors) that can interfere with the signal.
3.  **Too Many Devices:** Are there many devices connected to your Wi-Fi network? Each device uses a portion of your bandwidth. Try disconnecting some devices temporarily to see if your speed improves.
4.  **Wi-Fi Band:** Most modern routers broadcast on two frequencies: 2.4 GHz and 5 GHz.
    *   **2.4 GHz** has a wider range and penetrates walls better, but is generally slower and more prone to interference.
    *   **5 GHz** is much faster but has a shorter range and is more easily blocked by obstacles.
    *   If your router broadcasts both, try connecting to the 5 GHz network if you're close to the router.

---

### **Step 4: Check Your Device (Computer/Phone)**

Sometimes the problem isn't the internet itself, but the device you're using.

1.  **Restart Your Device:** Just like the router, a quick reboot of your computer, phone, or tablet can clear out any temporary software glitches.
2.  **Close Unnecessary Programs/Tabs:** If you have many applications running or dozens of browser tabs open, they can consume resources and slow down your device, making your internet *feel* slow.
3.  **Clear Browser Cache:** Over time, your web browser accumulates temporary files (cache and cookies) which can sometimes slow down browsing. Look in your browser's settings for an option to clear browsing data.
4.  **Check for Malware:** Malicious software can consume bandwidth and system resources. Run a scan with your antivirus software if you have one.
5.  **Try a Different Browser:** If one browser is slow, try another (e.g., if Chrome is slow, try Firefox or Edge) to see if the issue is browser-specific.

---

### **Step 5: Consider a Wired Connection (Ethernet)**

If possible, try connecting your computer directly to your router with an Ethernet cable.

*   If your speed dramatically improves when wired, the problem is likely with your Wi-Fi signal or router's wireless capabilities.
*   If the speed is still slow even with a wired connection, the issue is more likely with your modem, your ISP, or your internet line coming into your home.

---

### **Step 6: Contact Your Internet Service Provider (ISP)**

If you've tried all the above steps and your internet is still super slow, it's time to reach out to your ISP. They can:

*   **Check for outages** in your area.
*   **Run diagnostics on your line** remotely.
*   **Verify your modem's health.**
*   **Confirm your service plan** and ensure you're getting the speeds you're paying for.
*   **Schedule a technician visit** if necessary.

---

I know this is a lot of information, but taking these steps systematically will help us figure out what's going on.

Please let me know the answers to those initial questions, and what results you get from the speed test after trying the reboot. We'll get to the bottom of this!


In [38]:
queries = [
    "I was charged twice for my last bill, please fix this!",
    "Can I change my subscription plan?",
    "The website is not loading properly on my phone."
]

for q in queries:
    print(f"=== Query: {q} ===")
    display_result(run_customer_support(q))


=== Query: I was charged twice for my last bill, please fix this! ===
Negative



**🔖 Category:** This query falls under the **Billing** category.

It directly relates to charges, invoices, and payment discrepancies.
**💡 Sentiment:** Negative

**📝 Response:**  
Your query has been escalated to a human agent.


=== Query: Can I change my subscription plan? ===
**Neutral**

**Reasoning:** The query "Can I change my subscription plan?" is a straightforward request for information. It does not express any positive or negative sentiment towards the current plan, the service, or the desire to change. It's simply a functional question.



**🔖 Category:** This query falls under **Billing**.

Changing a subscription plan directly impacts the customer's charges, invoices, and overall financial relationship with the service.
**💡 Sentiment:** **Neutral**

**Reasoning:** The query "Can I change my subscription plan?" is a straightforward request for information. It does not express any positive or negative sentiment towards the current plan, the service, or the desire to change. It's simply a functional question.

**📝 Response:**  
Subject: Re: Can I change my subscription plan?

Hi there,

Absolutely, you can definitely change your subscription plan! We're happy to help you find the best fit for your needs.

Here's how you can usually do it:

1.  **Through Your Account Settings:** The quickest way is often by logging into your account on our website. Look for a section like "Account Settings," "Manage Subscription," "Billing," or "Plans." From there, you should see options to view other plans and make changes.
2.  **What to Expect:**
    *   **Plan Options:** You'll be able to see all available plans, their features, and pricing directly within that section.
    *   **Proration:** If you change your plan mid-billing cycle (especially when upgrading), any unused portion of your current plan will typically be prorated and applied as a credit to your new plan, or adjusted on your next bill.
    *   **Effective Date:** The change usually takes effect either immediately or at the start of your next billing cycle, depending on the specific plan change (e.g., upgrades often take effect immediately, while downgrades might take effect at the end of your current cycle).

If you encounter any issues, can't find the option, or would like to discuss which plan might be best for your specific requirements, please don't hesitate to reply to this email or call us directly at [Your Phone Number].

To help us assist you faster, if you reply, please include your account email address or customer ID.

We're here to ensure you have the best experience!

Best regards,

[Your Name/Company Name] Support Team


=== Query: The website is not loading properly on my phone. ===
Negative



**🔖 Category:** This query falls under the **Technical** category.

It describes a functional issue with the website's performance/display, which requires technical troubleshooting.
**💡 Sentiment:** Negative

**📝 Response:**  
Your query has been escalated to a human agent.
